In [1]:
from bs4 import BeautifulSoup as bs
import requests
from random import seed
import time
import json
import re
import pandas as pd

In [32]:
# Mini version de dataframe d'urls pour tester si le scraping fonctionne

# Il faudra gérer ensuite le volume conséquent de résumés à scraper
# On scrape pour chaque film, donc il faut extraire un dataframe d'urls seulement (3581 films dans la base de test) qu'on re-mergera avec la base test ensuite
# pour faire les moyennes des notes (labels)
test_mix = pd.read_csv('test_mix.csv')
data_mini_scrap = pd.DataFrame(pd.unique(test_mix['url']), columns=['url']).head(3)
data_mini_scrap

,url
0,https://www.imdb.com/title/tt0077699/
1,https://www.imdb.com/title/tt0463903/
2,https://www.imdb.com/title/tt0182576/


In [33]:
#On fabrique le squelette du dataframe que l'on va remplir au fur et à mesure du scrap
#On indique le nom des colonnes, qui sont les variables que l'on choisit de conserver
df = pd.DataFrame(columns=['url', 'name', 'alternateName', 'genre', 'description'])

In [34]:
#C'est une liste des objets inutiles dans le scrap des pages ; je retire aussi le contenu du dataframe
#qu'on a déjà (les votes)
superflu = ["@context", "@type", "image", "review", "trailer", "aggregateRating", 'contentRating', 'datePublished', 'actor', 'director',
                           'creator', 'Origine', 'Budget', 'duration', 'keywords' ]
#Celui-ci servira à retirer les images des scénaristes, etc
superflu2 = ['@type', 'url']

session_obj = requests.Session()


#On boucle sur chaque film qu'on considère
#Le compteur est cosmétique : il sert à nous rassurer sur le fait que tout se passe bien pendant le scrap
compteur = 0
for url in data_mini_scrap['url'] :
  compteur = compteur+1
  print(compteur)
  try: #On utilise un try except au cas où on aurait un problème sur une page :
    #on ne veut pas que l'exécution s'arrête après des heures de scrap
    time.sleep(0.01) #On ajoute un petit délai pour ne pas surcharger le site de requêtes
    response=session_obj.get(url, headers={"User-Agent": "Mozilla/5.0"}) #On se fait passer pour une session normale 
    html = response.content
    soup = bs(html, "html.parser")
    
    #Le bloc est composé de la partie de chaque page qui contient les informations utiles
    # On le transforme en dictionnaire
    bloc = soup.find("script", type="application/ld+json").string
    dictio = json.loads(bloc)
    
    #On retire dedans ce qui ne nous intéresse pas
    for inutile in superflu :
      dictio.pop(inutile, None)


    #On ajoute dans le dataframe la ligne qui correspond au film
    df = pd.concat([df, pd.DataFrame([dictio])], ignore_index=True)
  except:
    print('Erreur au rang : '+str(compteur))

1
2
3


In [35]:
df

,url,name,alternateName,genre,description
0,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...
1,https://www.imdb.com/title/tt0463903/,Be with Me,NaN,"[Drama, Romance]",&quot;Be with Me&quot; consists of three stori...
2,https://www.imdb.com/title/tt0182576/,Family Guy,Les Griffin,"[Animation, Comedy]","In a wacky Rhode Island town, a dysfunctional ..."


In [41]:
# Merge avec le dataset de test
test_descriptions = pd.merge(df, test_mix, on="url", how="inner")

In [42]:
test_descriptions

,url,name,alternateName,genre,description,Unnamed: 0,review,id_review,note,id_imdb_movie,note_bin,note_prec
0,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,0,This is without a doubt one of Neil Simon's be...,6742,10,tt0077699,1,Very Positive
1,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,237,The film isn't perfect by any means but despit...,3666,9,tt0077699,1,Very Positive
2,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,283,`Manna From Heaven' is a delightfully compelli...,10992,10,tt0077699,1,Very Positive
3,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,1357,The movie is more of a mockumentary of corrupt...,5440,8,tt0077699,1,Positive
4,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,5066,This movie was a pleasant surprise for me. In ...,6836,7,tt0077699,1,Positive
5,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,5229,It's initial premise is based on the American ...,6651,7,tt0077699,1,Positive
6,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,5259,"""Murder Over New York"" is fun, but not as good...",4525,8,tt0077699,1,Positive
7,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,13061,Three American lads are backpacking their way ...,1704,8,tt0077699,1,Positive
8,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,17444,"The last of the sequels,not counting Abbott an...",7105,8,tt0077699,1,Positive
9,https://www.imdb.com/title/tt0077699/,House Calls,Appelez-moi Docteur,"[Comedy, Drama, Romance]",Charley is a surgeon who&apos;s recently lost ...,19065,This kind of angst can only be inside a young ...,11278,10,tt0077699,1,Very Positive


In [45]:
test_descriptions.to_csv("test_descr.csv", index=True) 